# Analyzing Samsung Health Data

Concepts and methodologies for parsing, cleaning, and visualizing Samsung Health exports are taken from
[Analyzing Samsung Health Data](https://www.kaggle.com/code/kmader/analyzing-samsung-health-data/notebook#Samsung-Data) notebook by K. Scott Mader.

I have adapt that work to align with the latest update of the Samsung Health app, version 6.30.5.105, released on September 23, 2025.

## Extracting Data from Samsung Health

Exporting Personal Data (CSV/JSON) directly from the app:
1. Open The App in your phone.
2. Access the Menu tapping the three vertical dots (or a similar menu icon) usually located in the top-right corner.
3. Select Settings from the menu.
4. Look for and select the option `Download personal data` (or `Export data`).
5. The application will process and download the data to the `Samsung Health` subfolder located within your device's `Download` directory.

### File format
The exported data is typically provided in CSV (.csv) and JSON (.json) formats.

Next, we'll use Python code snippets to open and analyze the health data stored in CSV files.

## Setup and Function Definition

The first row always contains a metadata line.
The actual header appears in the next row.

Each data row may start with an empty leading column (a leading comma),
then pandas will assume that first column is the DataFrame index,
shifting all columns left so values end up under misaligned headers.

Curiously, that happens even if the header has a name for it.

In [1]:
# Use the Matplotlib inline magic command (if not already set)
%matplotlib inline
import os
import pandas as pd
from glob import glob
from IPython.display import display, Markdown

# The display function from IPython.display
# is essential in Jupyter Notebooks for rendering objects
# (like pandas DataFrames) in a visually appealing HTML format,
# rather than just raw text output.

samsung_dump_dir = 'samsunghealth_data'
samsung_csv_paths = glob(os.path.join(samsung_dump_dir, '*.csv'))
samsung_json_paths = glob(os.path.join(samsung_dump_dir, 'jsons', '*'))

# Define an anonymous function that reads CSV files
# using the pandas library with these specific arguments:
#   `skiprows=1`: ignore the first metadata line.
#   `index_col=False`: ensure the first column is treated as a regular data column.
sam_readcsv = lambda x: pd.read_csv(x, skiprows=1, index_col=False)

# Loads every specified CSV file into a single dictionary called `all_csv_df` (dictionary comprehension).
# The dictionary keys consist of the filename of the single CSV path.
all_csv_df = {os.path.basename(j): sam_readcsv(j) for j in samsung_csv_paths}


def display_list(names_list):
    """
    Display names list as the header row of a table.
    """
    # Create a temporary single-row DataFrame whose *columns* consist of the given names list.
    # This single row contains empty strings or placeholders, so it is NOT technically empty
    # suppressing "no rows to show" warning in Jupyter Notebooks.
    header_display_df = pd.DataFrame(columns=names_list, data=[['' for _ in names_list]])
    display(header_display_df)


def get_matching_columns(names_list, keyword):
    """
    Reads the names list and returns any column name
    that contains the specified keyword.
    """
    # Convert keyword to lowercase for case-insensitive checking
    keyword = keyword.lower()
    matched_columns = []
    for col in names_list:
        # Case-insensitive checking if the keyword is present in any column name.
        # If a column name consists of dot-separated components, only the last is checked.
        if keyword in col.split(".")[-1].strip().lower():
            matched_columns.append(col)    
    return matched_columns

## Process Samsung CSV files

In [2]:
print('Dump dir:', samsung_dump_dir)

print(len(samsung_csv_paths), 'csvs found')
print(len(samsung_json_paths), 'jsons found')

Dump dir: samsunghealth_data
52 csvs found
30 jsons found


### Display a small, random sample of the data from all CSVs

In [ ]:
# Start a loop to iterate through the key-value pairs of the `all_csv_df` dictionary:
#   k = The clean filename/key (e.g., `heart_rate.csv`).
#   v = The pandas DataFrame (the data itself).
for k, v in all_csv_df.items():
    # Print the name of the dataset and the total number of records it contains.
    #   v.shape[0] returns the number of rows in the DataFrame (the number of readings).
    print(k, 'readings:', v.shape[0])

    # Display a small, random sample of the data from the current DataFrame.
    # Select 2 random rows if the DataFrame has at least 3 rows, or just 1 row if it has 2 or fewer rows.
    display(v.sample(2 if v.shape[0]>2 else 1))

### Display DataFrame columns containing a specified keyword

In [2]:
# Replace 'target_keyword' with the word you are looking for (e.g., 'heart')
TARGET_KEYWORD = 'heart' 

banner = f"Target Keyword: '{TARGET_KEYWORD}'"
print(f"\n{banner}")
print("-" * len(banner))

# Start a loop to iterate through the key-value pairs of the `all_csv_df` dictionary:
#   k = The clean filename/key (e.g., `heart_rate.csv`).
#   v = The pandas DataFrame (the data itself).
found_cnt = 0
for k, v in all_csv_df.items():
    columns = get_matching_columns(v.columns, TARGET_KEYWORD)

    if len(columns) > 0:
        found_cnt = found_cnt + 1

        # Display the file name and matched count
        print(f'{k}: Found {len(columns)} matching column(s)')

        # Display the column names that contain the target keyword
        display_list(columns)

display(Markdown(f"### Found **{found_cnt}** CSV files matching the keyword '{TARGET_KEYWORD}'"))


Target Keyword: 'heart'
-----------------------
com.samsung.health.ecg.20251109104622.csv: Found 3 matching column(s)


,mean_heart_rate,max_heart_rate,min_heart_rate
0,,,


com.samsung.shealth.exercise.20251109104622.csv: Found 5 matching column(s)


,heart_rate_sample_count,heart_rate_deviceuuid,com.samsung.health.exercise.mean_heart_rate,com.samsung.health.exercise.max_heart_rate,com.samsung.health.exercise.min_heart_rate
0,,,,,


com.samsung.shealth.exercise.recovery_heart_rate.20251109104622.csv: Found 1 matching column(s)


...

com.samsung.shealth.tracker.heart_rate.20251109104622.csv: Found 2 matching column(s)


,com.samsung.health.heart_rate.heart_beat_count,com.samsung.health.heart_rate.heart_rate
0,,


com.samsung.shealth.tracker.oxygen_saturation.20251109104622.csv: Found 1 matching column(s)


,com.samsung.health.oxygen_saturation.heart_rate
0,


### Found **5** CSV files matching the keyword 'heart'

### Display the DataFrame headers from all CSVs

In [5]:
# Start a loop to iterate through the key-value pairs of the `all_csv_df` dictionary:
#   k = The clean filename/key (e.g., `heart_rate.csv`).
#   v = The pandas DataFrame (the data itself).
for k, v in all_csv_df.items():
    # Print the name of the dataset
    print(k)

    # Display dataset column names
    display_list(v.columns)

com.samsung.health.advanced_glycation_endproduct.20251018112612.csv


,create_sh_ver,measurement_result,percent,modify_sh_ver,update_time,create_time,score,deviceuuid,level_boundary,pkg_name,datauuid,day_time
0,,,,,,,,,,,,


com.samsung.health.advanced_glycation_endproduct.raw.20251018112612.csv


,create_sh_ver,start_time,binning_data,modify_sh_ver,update_time,create_time,version,time_offset,deviceuuid,pkg_name,end_time,datauuid
0,,,,,,,,,,,,


com.samsung.health.device_profile.20251018112612.csv


...

,create_sh_ver,step_count,binning_data,active_time,recommendation,modify_sh_ver,run_step_count,update_time,source_package_name,create_time,...,speed,distance,calorie,walk_step_count,deviceuuid,pkg_name,healthy_step,achievement,datauuid,day_time
0,,,,,,,,,,,...,,,,,,,,,,


com.samsung.shealth.tracker.pedometer_step_count.20251018112612.csv


,duration,version_code,run_step,walk_step,com.samsung.health.step_count.start_time,com.samsung.health.step_count.sample_position_type,com.samsung.health.step_count.custom,com.samsung.health.step_count.update_time,com.samsung.health.step_count.create_time,com.samsung.health.step_count.count,com.samsung.health.step_count.speed,com.samsung.health.step_count.distance,com.samsung.health.step_count.calorie,com.samsung.health.step_count.time_offset,com.samsung.health.step_count.deviceuuid,com.samsung.health.step_count.pkg_name,com.samsung.health.step_count.end_time,com.samsung.health.step_count.datauuid
0,,,,,,,,,,,,,,,,,,


com.samsung.shealth.vitality_score.20251018112612.csv


,activity_level,activity_score,create_sh_ver,active_time_scale,activity_balance_short_term_activity,activity_balance_long_term_activity,shr_balance_scale,mvpa_time_optimal_range_max,mvpa_time_optimal_range_min,max_hr,...,active_time_range_max,active_time_range_min,shrv_baseline_max,shrv_baseline_min,datauuid,prev_shr_avg,day_time,shrv_penalty,activity_balance_optimal_range_max,activity_balance_optimal_range_min
0,,,,,,,,,,,...,,,,,,,,,,
